In [ ]:
"""
Module description:

Run script for the emission preparation workflow.
Script prepare point source and area emission input for DALES model
Input: CSV files of point source and area emission data of GHGs (provided by RIVM) (1x1 or 5x5 km resolution)
+ CRS household data to refine consu (SNAP2) from https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/kaart-van-100-meter-bij-100-meter-met-statistieken
and NOx emission data from traffic to refine traffic GHG emissions (SNAP7) (data prepared by Dat.MOBILITY company, ask RIVM for these data)

Output: NetCDF files of explicit model input.

The run time of the workflow strongly depends upon the domain size and resolution
Reminder: currently this workflow is suitable to transfer emissions of 1x1km (5x5km too) to the demain resolution and 
from RD to HARM coordinates. 
If you some need other or additional setting, feel free to fit scripts of this workflow to your subject.

Author: Dr. Arseni Doyennel
Contact Email: a.doyennel@vu.nl
"""
#Import all modules for workflow
from point_source_explicit_input_netcdf import Pointsource_input_preparation
from ruisdael_area_residuals import Emission_initial_process_get_residuals
from ruisdael_area_csv2gpkg import EmissionConverter
from consu_ruisdael_area_csv2gpkg import EmissionConverter_consu
from ruisdael_area_RD2HARM import AreaEmissionsReprojection
from consu_ruisdael_area_RD2HARM import AreaEmissionsReprojection_Consu
from traffic_ruisdael_area_RD2HARM import AreaEmissionsReprojection_Traffic
from ruisdael_area_gpkg2csv import EmissionPreparation
from ruisdael_area_csv2netc import CSV_to_NCDF_emiss_convertor
from ruisdael_area_total_static import Emission_3Dfield_creation
from create_hourly_emissions_3D import Final_3D_input

if __name__ == "__main__":
    
    # Instantiate classes and call the methods:
    #point sources model input preparation:
    PSEP = Pointsource_input_preparation()
    df, df_org = PSEP.identify_emission_categories()
    df_selection_non_rem_full, df_gapfilled_full = PSEP.prepare_emission_data(df)
    #PSEP.plot_emission_data(df_selection_non_rem_full, df_gapfilled_full)
    PSEP.prepare_final_input(df_gapfilled_full)

    
    # area emissions residuals:
    emission_init_prep = Emission_initial_process_get_residuals()
    df_point, df_point_private = emission_init_prep.read_point()
    df_point_2 = emission_init_prep.preprocess_point_data(df_point, df_point_private)
    emission_init_prep.save_points_harm(df_point_2)
    field_source, df_map, x, y = emission_init_prep.substract_point_from_map(df_point_2)
    field_subdoelgroep_source_roads = emission_init_prep.populate_road_emissions(df_map, x, y)
    emission_init_prep.save_data(field_source, field_subdoelgroep_source_roads, x, y)
    emission_init_prep.refine_and_save_snap2_data(field_source)
    
    
    #CSV to GPKG:
    # Instantiate the EmissionConverter class
    EC = EmissionConverter()
    # Run the conversion process
    EC.convert_emissions()
    # Merge the generated GeoPackages and remove intermediate files
    EC.merge_and_remove_data()
    
    
    #CSV to GPKG (consu):
    EC = EmissionConverter_consu()
    EC.prepare_emission_data()
    EC.merge_and_remove_data()
    
    
    #RD2HARM:
    # Instantiate the AreaEmissionsReprojection class
    AER=AreaEmissionsReprojection()
    # Run the reprojection and downscaling process
    AER.reproject_emissions()
    
    #RD2HARM (consu):
    # Instantiate the AreaEmissionsReprojection class
    AER=AreaEmissionsReprojection_Consu()
    # Run the reprojection and downscaling process
    AER.reproject_emissions()
    
    
    #RD2HARM (traffic):
    # Instantiate the AreaEmissionsReprojection class
    AER=AreaEmissionsReprojection_Traffic()
    # Run the reprojection and downscaling process
    AER.reproject_emission_and_refine()
    
    
    #GPKG to CSV:
    emission_prep = EmissionPreparation()
    emission_prep.prepare_emission_data()

    
    
    #CSV to NetCDF:
    # Instantiate the class
    CSV2NCDF = CSV_to_NCDF_emiss_convertor()
    CSV2NCDF.prep_ncdfs()
    
    
    #static 3-D:
    E_3D = Emission_3Dfield_creation()
    E_3D.process_emissions()
    
    
    #Final 3-D area emiss input preparation per hour:
    emission_final_prep = Final_3D_input()
    emission_final_prep.create_output_directory()
    emission_final_prep.create_settings_file()
    emission_final_prep.create_emissions()
    
    